<a href="https://colab.research.google.com/github/koheikobayashi/bertsum/blob/main/BERTsum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 深層学習モデルを用いたQiita記事のタイトル自動生成

## 依存ライブラリのインストール

In [1]:
!pip install -qU torch==1.7.1 torchtext==0.8.0 torchvision==0.8.2
!pip install -q transformers==4.2.2 sentencepiece

     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 6.9 MB 52.4 MB/s 
     |████████████████████████████████| 12.8 MB 60.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.7.1 which is incompatible.
     |████████████████████████████████| 1.8 MB 29.3 MB/s 
     |████████████████████████████████| 1.3 MB 59.5 MB/s 
     |████████████████████████████████| 2.9 MB 50.3 MB/s 
     |████████████████████████████████| 880 kB 59.0 MB/s 


## 学習済みモデルの読み込み

この深層学習モデルは、日本語コーパスを用いて事前学習したT5モデルを今回のタイトル生成タスク向けに転移学習したものです。  

T5とは: https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part7.html

今回のモデルの入出力は次の形式になっています。

- **入力**: "body: {body}"をトークナイズしたもの
- **出力**: "title: {title}"をトークナイズしたもの

ここで、{body}はMarkdown形式の記事本文を前処理した文字列、{title}は生成されたタイトルです。


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import AdamW,get_linear_schedule_with_warmup

# 学習済みモデルをHugging Face model hubからダウンロードする
model_dir_name = "sonoisa/t5-qiita-title-generation"

# トークナイザー（SentencePiece）
tokenizer = T5Tokenizer.from_pretrained(model_dir_name, is_fast=True)

# 学習済みモデル
trained_model = T5ForConditionalGeneration.from_pretrained(model_dir_name)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
  trained_model.cuda()

Downloading:   0%|          | 0.00/804k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

## 前処理の定義

### 文字列の正規化

表記揺れを減らします。今回は[neologdの正規化処理](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja)を一部改変したものを利用します。
処理の詳細はリンク先を参照してください。



In [3]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

### Markdownのクリーニング

タイトルを考える上で関係のなさそうな文章を削る処理を行います。  
以下のノイズとなるデータを削除し、タブや改行を空白文字にしたり、文字を小文字に揃える等の処理を行います。

- ソースコード
- URLやリンク
- 画像

現状、img以外のHTML要素を残していますが、タイトルに関係なさそうな要素を削ると精度が上がるかもしれません。

In [4]:
import re

CODE_PATTERN = re.compile(r"```.*?```", re.MULTILINE | re.DOTALL)
LINK_PATTERN = re.compile(r"!?\[([^\]\)]+)\]\([^\)]+\)")
IMG_PATTERN = re.compile(r"<img[^>]*>")
URL_PATTERN = re.compile(r"(http|ftp)s?://[^\s]+")
NEWLINES_PATTERN = re.compile(r"(\s*\n\s*)+")

def clean_markdown(markdown_text):
    markdown_text = CODE_PATTERN.sub(r"", markdown_text)
    markdown_text = LINK_PATTERN.sub(r"\1", markdown_text)
    markdown_text = IMG_PATTERN.sub(r"", markdown_text)
    markdown_text = URL_PATTERN.sub(r"", markdown_text)
    markdown_text = NEWLINES_PATTERN.sub(r"\n", markdown_text)
    markdown_text = markdown_text.replace("`", "")
    return markdown_text

def normalize_text(markdown_text):
    markdown_text = clean_markdown(markdown_text)
    markdown_text = markdown_text.replace("\t", " ")
    markdown_text = normalize_neologd(markdown_text).lower()
    markdown_text = markdown_text.replace("\n", " ")
    return markdown_text

def preprocess_qiita_body(markdown_text):
    return "body: " + normalize_text(markdown_text)[:4000]

## 後処理の定義

モデルの出力は"title: {title}"という形式ですので、後処理として余計な"title: "を削除するようにします。

これで必要な前処理、後処理を定義できました。

In [5]:
def postprocess_title(title):
  return re.sub(r"^title: ", "", title)

## タイトル生成の対象となる記事本文の定義

今回は下記のような記事に対するタイトル生成を試してみます。  
中身を色々変えて試してみてください。

In [6]:
qiita_body = """
AIの進歩はすごいですね。

今回は深層学習を用いて、記事（Qiita）のタイトルを自動生成してくれるAIさんを試作してみました。
この実験は自然言語処理について新人さんに教えるためのハンズオンネタを探索する一環で行ったものになります。

作ったAIは、Qiitaの記事本文（少し前処理したテキスト）を与えると、適したタイトル文字列を作文して返してくれるというものです。

なお、学習データは（2019年頃に）Qiitaの殿堂を入り口にして、評価の高い記事（いいねが50個以上）をスクレイピングしたものを使いました。
つまりヒットした記事のタイトルの付け方を学んだAIであるといえます。

* もう少し詳細:
  * 学習データの例:
    * 入力: "body: hiveqlではスピードに難を感じていたため、私もprestoを使い始めました。 mysqlやhiveで使っていたクエリ..."
    * 出力: "title: hadoop利用者ならきっと知ってる、hive/prestoクエリ関数の挙動の違い"
  * 学習方法: 独自に作った日本語T5の事前学習モデルをこの学習データを用いて転移学習

以下、結果（抜粋）です。generatedが生成されたもの、actualが人が付けたタイトルです。
"""

この文章に対して前処理をすると次のようになります。これをトークナイズしたものがモデルの入力になります。

In [7]:
preprocess_qiita_body(qiita_body)

'body: aiの進歩はすごいですね。 今回は深層学習を用いて、記事(qiita)のタイトルを自動生成してくれるaiさんを試作してみました。 この実験は自然言語処理について新人さんに教えるためのハンズオンネタを探索する一環で行ったものになります。 作ったaiは、qiitaの記事本文(少し前処理したテキスト)を与えると、適したタイトル文字列を作文して返してくれるというものです。 なお、学習データは(2019年頃に)qiitaの殿堂を入り口にして、評価の高い記事(いいねが50個以上)をスクレイピングしたものを使いました。 つまりヒットした記事のタイトルの付け方を学んだaiであるといえます。 *もう少し詳細: *学習データの例: *入力:\xa0"body:\xa0hiveqlではスピードに難を感じていたため、私もprestoを使い始めました。\xa0mysqlやhiveで使っていたクエリ..." *出力:\xa0"title:\xa0hadoop利用者ならきっと知ってる、hive/prestoクエリ関数の挙動の違い" *学習方法:\xa0独自に作った日本語t5の事前学習モデルをこの学習データを用いて転移学習 以下、結果(抜粋)です。generatedが生成されたもの、actualが人が付けたタイトルです。'

## タイトルの自動生成を実行

記事に合うタイトルを10個、自動生成してみます。

以下のコードではタイトルの多様性を生むために色々generateメソッドのパラメータを設定しています。パラメータの詳細は下記リンク先を参照してください。

https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate

In [8]:
MAX_SOURCE_LENGTH = 512  # 入力される記事本文の最大トークン数
MAX_TARGET_LENGTH = 64   # 生成されるタイトルの最大トークン数

# 推論モード設定
trained_model.eval()

# 前処理とトークナイズを行う
inputs = [preprocess_qiita_body(qiita_body)]
batch = tokenizer.batch_encode_plus(
    inputs, max_length=MAX_SOURCE_LENGTH, truncation=True, 
    padding="longest", return_tensors="pt")

input_ids = batch['input_ids']
input_mask = batch['attention_mask']
if USE_GPU:
  input_ids = input_ids.cuda()
  input_mask = input_mask.cuda()

# 生成処理を行う
outputs = trained_model.generate(
    input_ids=input_ids, attention_mask=input_mask, 
    max_length=MAX_TARGET_LENGTH,
    return_dict_in_generate=True, output_scores=True,
    temperature=1.0,            # 生成にランダム性を入れる温度パラメータ
    num_beams=10,               # ビームサーチの探索幅
    diversity_penalty=1.0,      # 生成結果の多様性を生み出すためのペナルティ
    num_beam_groups=10,         # ビームサーチのグループ数
    num_return_sequences=10,    # 生成する文の数
    repetition_penalty=1.5,     # 同じ文の繰り返し（モード崩壊）へのペナルティ
)

# 生成されたトークン列を文字列に変換する
generated_titles = [tokenizer.decode(ids, skip_special_tokens=True, 
                                     clean_up_tokenization_spaces=False) 
                    for ids in outputs.sequences]

# 生成されたタイトルを表示する
for i, title in enumerate(generated_titles):
  print(f"{i+1:2}. {postprocess_title(title)}")

 1. 深層学習でqiitaのタイトルを自動生成してくれるaiを試作した
 2. aiの進化はすごい。記事タイトル自動生成のaiを試作した
 3. ディープラーニングで記事のタイトルを自動生成してくれるaiを試作した
 4. ディープラーニングで記事のタイトルを自動生成してくれるaiを試作した。
 5. aiがすごい勢いで記事のタイトルを自動生成するaiを試作した
 6. 【人工知能】深層学習で「記事タイトルを自動生成」する
 7. deep learningでqiitaのタイトルを自動生成するaiを作ってみた
 8. deep learningでqiitaのタイトルを自動生成するaiを作ってみた。
 9. 「記事のタイトルを自動生成してくれるai」を作ってみた。(結果)
10. 「記事のタイトルを自動生成してくれるai」を作ってみた
